# QUANTUM PREDICTION USING NEURAL NETWORKS

In [1]:
#!pip install qiskit_algorithms
#!pip install qiskit-machine-learning==0.7.1
#!pip install 'qiskit-machine-learning[torch]'
#!pip install --upgrade pip

## IMPORTS

In [2]:
from qiskit.circuit.library import RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.circuit.library import QNNCircuit

import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
algorithm_globals.random_seed = 42

## READ AND PREPARE

In [3]:
data = pd.read_csv("data/train.csv")
data.head()

data.loc[(data["Pclass"] == 1) & (data["Survived"] == 1) & (data["Sex"] == "female"), "Age"] = 38.0
data.loc[(data["Pclass"] == 2) & (data["Survived"] == 0) & (data["Sex"] == "female"), "Age"] = 26.0
data.loc[(data["Pclass"] == 2) & (data["Survived"] == 1) & (data["Sex"] == "female"), "Age"] = 17.0

data.loc[(data["Pclass"] == 3) & (data["Survived"] == 0) & (data["Sex"] == "female"), "Age"] = 8.0
data.loc[(data["Pclass"] == 3) & (data["Survived"] == 1) & (data["Sex"] == "female"), "Age"] = 27.0

data.loc[(data["Pclass"] == 1) & (data["Survived"] == 1) & (data["Sex"] == "male"), "Age"] = 23.0
data.loc[(data["Pclass"] == 1) & (data["Survived"] == 0) & (data["Sex"] == "male"), "Age"] = 19.0

data.loc[(data["Pclass"] == 2) & (data["Survived"] == 1) & (data["Sex"] == "male"), "Age"] = 62.0
data.loc[(data["Pclass"] == 2) & (data["Survived"] == 0) & (data["Sex"] == "male"), "Age"] = 35.0

data.loc[(data["Pclass"] == 3) & (data["Survived"] == 1) & (data["Sex"] == "male"), "Age"] = 12.0
data.loc[(data["Pclass"] == 3) & (data["Survived"] == 0) & (data["Sex"] == "male"), "Age"] = 35.0

data.loc[data["Sex"] == "male", "Sex"] = 1
data.loc[data["Sex"] == "female", "Sex"] = 0

data1 = data[["Pclass", "Sex", "Age", "Parch", "Fare", "Survived"]].copy()

df_train = data1.sample(frac=0.89, random_state=0)
df_valid = data1.drop(df_train.index)

X_train = df_train.drop('Survived', axis=1)
X_valid = df_valid.drop('Survived', axis=1)
y_train = df_train['Survived']
y_valid = df_valid['Survived']

scaler = StandardScaler()
X_train  = scaler.fit_transform(X_train)
X_valid  = scaler.fit_transform(X_valid)


# QUANTUM NEURAL NETWORK
## This code sets up a quantum neural network for a binary classification task. The QNN is trained as a classifier using the specified optimizer.

In [4]:
#Function to monitor iterations
def callback_graph(weights, obj_func_eval):
    progress_bar.update(1)    

num_inputs = X_train.shape[1]
max_iter = 7
progress_bar = tqdm(total=max_iter, desc="Iterations", unit="iteration") 
# Define a Quantum Neural Network (QNN) circuit using RealAmplitudes as the ansatz
qc = QNNCircuit(ansatz=RealAmplitudes(num_inputs, reps=1))

# Define a parity function that takes an integer 'x' and returns the parity (odd or even) of the number of set bits (1s) in its binary representation
def parity(x):    
    return "{:b}".format(x).count("1") % 2

# Set the output shape for the Quantum Neural Network (QNN)
output_shape = 2 

# Create a SamplerQNN, which is a quantum neural network with a specified circuit, interpretation function, and output shape
sampler_qnn = SamplerQNN(
    circuit=qc,
    interpret=parity,
    output_shape=output_shape,
)

# Create a NeuralNetworkClassifier using the SamplerQNN as the neural network and COBYLA as the optimizer
# The classifier is designed for binary classification tasks, so 'output_shape' is set to 2
NNC_model = NeuralNetworkClassifier(
    neural_network=sampler_qnn, optimizer=COBYLA(maxiter=max_iter), callback=callback_graph
)

NNC_model.fit(X_train , y_train)

progress_bar.close()
plt.show()

y_predict = NNC_model.predict(X_valid)
accuracy = accuracy_score(y_predict, y_valid)
print("Accuracy: ", accuracy)

Iterations: 100%|██████████| 7/7 [20:03<00:00, 171.94s/iteration]


Accuracy:  0.4897959183673469


## Test with the entire dataset

In [5]:
test = data1.drop('Survived', axis=1)
test  = scaler.fit_transform(test)
y_predict_all = NNC_model.predict(test)
data1['Predict'] = y_predict_all
data1


,Pclass,Sex,Age,Parch,Fare,Survived,Predict
0,3,1,35.0,0,7.2500,0,0
1,1,0,38.0,0,71.2833,1,1
2,3,0,27.0,0,7.9250,1,1
3,1,0,38.0,0,53.1000,1,0
4,3,1,35.0,0,8.0500,0,0
...,...,...,...,...,...,...,...
886,2,1,35.0,0,13.0000,0,1
887,1,0,38.0,0,30.0000,1,1
888,3,0,8.0,2,23.4500,0,0
889,1,1,23.0,0,30.0000,1,0


## Show Accuracy

In [6]:
accuracy = accuracy_score(data1["Survived"], data1["Predict"])
print(accuracy)

0.6767676767676768
